In [1]:
sys.path.insert(1,r'F:\Users\Kaggle\bulian')

In [2]:
import os,sys,torch
import pandas as pd

c:\Users\anaconda\envs\BulianAI\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from bulian.Tabular.synthesizers import TwinSynthesizer,PrivateTwinSynthesizer
from bulian.metrics import *
from bulian.metrics.reports import *
from bulian.metrics import compute_metrics
from bulian.metrics.single_table import SingleTableMetric
from bulian.metrics.single_table import *

c:\Users\anaconda\envs\BulianAI\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [5]:
discrete_columns = [
    "workclass",
    "education",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "native-country",
    "income",
]

In [6]:
import torch
torch.__version__

'1.12.0'

In [7]:
torch.cuda.is_available()

True

In [8]:
!nvidia-smi

Tue Jul 19 18:33:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.79       Driver Version: 511.79       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   47C    P8    24W / 370W |    820MiB / 24576MiB |     13%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

###### 

## Normal API: Non-privately differentiable synthesizer

In [9]:
data = pd.read_csv(r"F:\Users\Kaggle\bulian\examples\csv\adult.csv")

data = data.sample(1000).reset_index(drop=True)

In [10]:
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,21,Private,197387,HS-grad,9,Never-married,Handlers-cleaners,Own-child,White,Male,0,0,20,United-States,<=50K
1,26,Private,213081,HS-grad,9,Never-married,Adm-clerical,Unmarried,Black,Female,0,0,40,Jamaica,<=50K
2,23,Private,236601,Some-college,10,Never-married,Tech-support,Not-in-family,White,Male,0,0,48,United-States,<=50K
3,74,Self-emp-inc,228075,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,20051,0,25,United-States,>50K
4,25,Private,323139,Bachelors,13,Never-married,Handlers-cleaners,Own-child,White,Male,0,0,35,United-States,<=50K


In [11]:
synth = TwinSynthesizer(batch_size=200,device='cuda')   ### cpu else cuda

In [12]:
synth.fit(data=data,epochs=2,discrete_columns=discrete_columns)

Epoch: [0]  [0/4]  eta: 0:00:04  loss_g: 2.1354 (2.1354)  loss_d: -0.0060 (-0.0060)  loss: 2.1294 (2.1294)  time: 0.8920  data: 0.0000  max mem: 20
Epoch: [0]  [4/4]  eta: 0:00:00  loss_g: 2.0645 (2.0658)  loss_d: -0.0356 (-0.0492)  loss: 1.9716 (2.0166)  time: 0.2006  data: 0.0000  max mem: 24
Epoch: [0] Total time: 0:00:01
Epoch: [1]  [0/4]  eta: 0:00:00  loss_g: 2.0624 (2.0624)  loss_d: -0.0928 (-0.0928)  loss: 1.9696 (1.9696)  time: 0.0280  data: 0.0000  max mem: 24
Epoch: [1]  [4/4]  eta: 0:00:00  loss_g: 2.0892 (2.0997)  loss_d: -0.1620 (-0.1585)  loss: 1.9292 (1.9411)  time: 0.0302  data: 0.0000  max mem: 24
Epoch: [1] Total time: 0:00:00


In [13]:
sample = synth.sample(1000)

In [14]:
sample

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,41,Local-gov,266016,Assoc-voc,12,Widowed,?,Not-in-family,White,Female,2450,2113,39,Italy,>50K
1,28,Local-gov,217711,Bachelors,5,Divorced,Sales,Own-child,Other,Male,5652,1910,37,Canada,<=50K
2,31,Private,162062,9th,4,Married-civ-spouse,Farming-fishing,Own-child,Black,Male,-270,2074,34,Mexico,>50K
3,40,Federal-gov,119796,11th,6,Married-civ-spouse,Craft-repair,Husband,Other,Male,-202,2033,49,India,>50K
4,36,Private,273812,Bachelors,6,Never-married,Other-service,Husband,White,Female,730,2565,47,Thailand,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,50,Self-emp-inc,295388,Masters,10,Never-married,Handlers-cleaners,Own-child,White,Male,4953,21,40,Japan,>50K
996,22,Federal-gov,243157,Masters,15,Married-AF-spouse,Craft-repair,Wife,White,Male,8812,1419,42,Vietnam,>50K
997,42,?,183446,5th-6th,14,Married-AF-spouse,Sales,Not-in-family,White,Male,-84,2765,5,Japan,>50K
998,43,Self-emp-inc,204233,7th-8th,4,Never-married,Protective-serv,Other-relative,Asian-Pac-Islander,Female,4277,12,40,Hungary,>50K


In [15]:
metrics = SingleTableMetric.get_subclasses()
numeric_features = ['capital-gain','capital-loss','hours-per-week']
discrete_columns = [
    "workclass",
    "education",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "native-country",
    "income",
]

In [16]:
metrics

{'BNLogLikelihood': bulian.metrics.single_table.bayesian_network.BNLogLikelihood,
 'LogisticDetection': bulian.metrics.single_table.detection.sklearn.LogisticDetection,
 'SVCDetection': bulian.metrics.single_table.detection.sklearn.SVCDetection,
 'BinaryDecisionTreeClassifier': bulian.metrics.single_table.efficacy.binary.BinaryDecisionTreeClassifier,
 'BinaryAdaBoostClassifier': bulian.metrics.single_table.efficacy.binary.BinaryAdaBoostClassifier,
 'BinaryLogisticRegression': bulian.metrics.single_table.efficacy.binary.BinaryLogisticRegression,
 'BinaryMLPClassifier': bulian.metrics.single_table.efficacy.binary.BinaryMLPClassifier,
 'MulticlassDecisionTreeClassifier': bulian.metrics.single_table.efficacy.multiclass.MulticlassDecisionTreeClassifier,
 'MulticlassMLPClassifier': bulian.metrics.single_table.efficacy.multiclass.MulticlassMLPClassifier,
 'LinearRegression': bulian.metrics.single_table.efficacy.regression.LinearRegression,
 'MLPRegressor': bulian.metrics.single_table.efficacy

### Report with all metrics

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features, key_fields=['age','workclass','education'],sensitive_fields = ['income'],target='income')

#### Report with privacy metrics

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features, key_fields=['age','workclass','education'],sensitive_fields = ['income'])

#### Report without privacy metrics, but includes ML efficacy stuff

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features,target='income')

#### Report without privacy metrics and without ML efficacy stuff

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features)

#### Save model to disk

In [ ]:
synth.save('F:/Users/Kaggle/wids/NormalAPI.pth')

#### 

## Adversarial API: Non-privately differentiable synthesizer

In [ ]:
synth = TwinSynthesizer(batch_size=200,device='cuda')   ### cpu else cuda

In [ ]:
synth.fit_adversarial(data=data,epochs=2,discrete_columns=discrete_columns,test_pct=0.33)

In [ ]:
sample = synth.sample_adversarial(data,1000)

In [ ]:
metrics = SingleTableMetric.get_subclasses()
numeric_features = ['capital-gain','capital-loss','hours-per-week']
discrete_columns = [
    "workclass",
    "education",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "native-country",
    "income",
]

#### Report with privacy metrics

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features, key_fields=['age','workclass','education'],sensitive_fields = ['income'])

#### Report without privacy metrics, but includes ML efficacy stuff

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features,target='income')

#### Report without privacy metrics and without ML efficacy stuff

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features)

#### Save model to disk

In [ ]:
synth.save('F:/Users/Kaggle/wids/AdversarialAPI.pth')

#### 

## Normal API: Privately differentiable synthesizer

In [ ]:
synth = PrivateTwinSynthesizer(epsilon=0.1,batch_size=64,device='cuda')   ### cpu else cuda

In [ ]:
synth.fit(data=data,discrete_columns=discrete_columns,update_epsilon=1)

In [ ]:
sample = synth.sample(1000)

In [ ]:
metrics = SingleTableMetric.get_subclasses()
numeric_features = ['capital-gain','capital-loss','hours-per-week']
discrete_columns = [
    "workclass",
    "education",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "native-country",
    "income",
]

#### Report with privacy metrics

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features, key_fields=['age','workclass','education'],sensitive_fields = ['income'])

#### Report without privacy metrics, but includes ML efficacy stuff

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features,target='income')

#### Report without privacy metrics and without ML efficacy stuff

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features)

#### Save model to disk

In [ ]:
synth.save('F:/Users/Kaggle/wids/PrivateModelNormalAPI.pth')

#### 

## Adversarial API: Privately differentiable synthesizer

In [ ]:
synth = PrivateTwinSynthesizer(epsilon=0.1,batch_size=64,device='cuda')   ### cpu else cuda

In [ ]:
synth.fit_adversarial(data=data,discrete_columns=discrete_columns,update_epsilon=1)

In [ ]:
sample = synth.sample_adversarial(data,1000)

In [ ]:
metrics = SingleTableMetric.get_subclasses()
numeric_features = ['capital-gain','capital-loss','hours-per-week']
discrete_columns = [
    "workclass",
    "education",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "native-country",
    "income",
]

#### Report with privacy metrics

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features, key_fields=['age','workclass','education'],sensitive_fields = ['income'])

#### Report without privacy metrics, but includes ML efficacy stuff

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features,target='income')

#### Report without privacy metrics and without ML efficacy stuff

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features)

### Want all 5 metrics together?

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features, key_fields=['age','workclass','education'],sensitive_fields = ['income'],target='income')

#### Save model to disk

In [ ]:
synth.save('F:/Users/Kaggle/wids/PrivateModelAdvAPI.pth')

#### 

### Fin ###